In [8]:
filename = "new_train_labels.csv"
import pandas as pd
from tqdm import tqdm
df = pd.read_csv(filename, encoding='utf-8')

df["images"]=""
from tqdm import tqdm 
for i in tqdm(range(len(df))):
    if i == 0: 
        continue
    df["images"][i] = str(i)+".jpg"
print(df.head())

  0%|          | 0/125973 [00:00<?, ?it/s]C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 125973/125973 [1:01:06<00:00, 34.36it/s]


   label images
0      0       
1      0  1.jpg
2      1  2.jpg
3      0  3.jpg
4      0  4.jpg


In [9]:
df.tail() 

,label,images
125968,1,125968.jpg
125969,0,125969.jpg
125970,0,125970.jpg
125971,1,125971.jpg
125972,0,125972.jpg


In [3]:
imagefiles_list = []
from PIL import Image
import numpy as np

for i in tqdm(range(len(df))):
    if i == 0: 
        continue
    imagefiles_list.append(df['images'][i])
    
size = (8,8)
x = np.array([np.array(Image.open("./img_data_rev/img_data/"+fname).resize(size).convert('RGB')) for fname in tqdm(imagefiles_list)])
print(x.shape)

x.dump("train_features.pkl")

100%|██████████| 125972/125972 [05:27<00:00, 439.17it/s]


(125972, 8, 8, 3)


In [18]:
#labels = pd.get_dummies(df["label"][1:]).as_matrix()
import numpy as np
labels = df["label"][1:].as_matrix()
print(labels.shape)
labels.dump("train_labels.pkl")

(125972,)


In [19]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv2D,MaxPooling2D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
import time
from keras.utils import np_utils
from sklearn import preprocessing as pp

import h5py

X_train = np.load("train_features.pkl")
Y_train = np.load("train_labels.pkl")



cnn2d_1 = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])
print(cnn2d_1.summary())

C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 64)          1792      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 64)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [13]:
test_df = pd.read_csv("test_labels.csv", encoding='utf-8')
test_df["images"]=""
from tqdm import tqdm 
for i in tqdm(range(len(test_df))):
    if i == 0: 
        continue
    test_df["images"][i] = str(i)+".jpg"
test_imagefiles_list = []
for i in tqdm(range(len(test_df))):
    if i == 0: 
        continue 
    test_imagefiles_list.append(test_df['images'][i])
    
size = (8,8)
test_x = np.array([np.array(Image.open("./img_kddtest+/img_kddtest+/"+fname).resize(size).convert('RGB')) for fname in tqdm(test_imagefiles_list)])
print(test_x.shape)

test_x.dump("test_features.pkl")

test_labels = pd.get_dummies(test_df["label"]).as_matrix()

print(test_labels.shape)
test_labels.dump("test_labels.pkl")


100%|██████████| 22543/22543 [00:05<00:00, 3806.05it/s]

(22543, 8, 8, 3)
(22544, 38)


In [20]:
import time
cnn2d_1.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn2d_1.fit(X_train, Y_train, batch_size=64,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn2d_1.to_json()
with open("NSL_KDD_Train.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn2d_1.save_weights("NSL_KDD_Train_weights.h5")

Epoch 1/10
125972/125972 [==============================] - 26s 208us/step - loss: 7.5023 - acc: 0.5345
Epoch 2/10
125972/125972 [==============================] - 25s 197us/step - loss: 7.5017 - acc: 0.5346
Epoch 3/10
125972/125972 [==============================] - 25s 195us/step - loss: 7.5017 - acc: 0.5346
Epoch 4/10
125972/125972 [==============================] - 25s 195us/step - loss: 7.5017 - acc: 0.5346
Epoch 5/10
125972/125972 [==============================] - 25s 196us/step - loss: 7.5017 - acc: 0.5346
Epoch 6/10
125972/125972 [==============================] - 25s 196us/step - loss: 7.5017 - acc: 0.5346
Epoch 7/10
125972/125972 [==============================] - 25s 195us/step - loss: 7.5017 - acc: 0.5346
Epoch 8/10
125972/125972 [==============================] - 24s 193us/step - loss: 7.5017 - acc: 0.5346
Epoch 9/10
125972/125972 [==============================] - 3469s 28ms/step - loss: 7.5017 - acc: 0.5346
Epoch 10/10
125972/125972 [==============================] - 53

In [ ]:
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_01.png')